Data crawling

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

class_lst = ['Barbera', 'Cabernet', 'Chardonnay', 'Merlot', 'Montepulciano', 'Moscato', 'Pinot Grigio', 'Sangiovese', 'Sauvignon', 'Syrah', 'CiuCiu']
#NUM = 10000

for i, grape_type in enumerate(class_lst):#
    if i != len(class_lst)-1:
        continue
    if grape_type == 'CiuCiu':
        text_query = "wine, " + grape_type + ' lang:en'
    else:
        text_query = "wine, " + grape_type + ' lang:en since:2015-01-01 until:2022-09-30'
    result = []
    for n, tweet in enumerate(sntwitter.TwitterSearchScraper(text_query).get_items()):
        '''if n > NUM:
            break'''
        print('============================================')
        print(n)
        print(tweet.id)
        print(tweet.rawContent)
        print(tweet.date)
        print(tweet.user.username)
        print(tweet.user.location)
        print(tweet.replyCount)
        print(tweet.retweetCount)
        print(tweet.likeCount)
        print(tweet.quoteCount)
        result.append({ 'id':tweet.id,
                        'text':tweet.rawContent,
                        'time':tweet.date,
                        'user_location':tweet.user.location,
                        'username':tweet.user.username,
                        'replyCount':tweet.replyCount,
                        'retweetCount':tweet.retweetCount,
                        'likeCount':tweet.likeCount,
                        'quoteCount':tweet.quoteCount,
                        'class':grape_type
            })

    df = pd.DataFrame(result)
    df.to_csv(grape_type+'.csv',index=False)

Data Preprocess

In [ ]:
# import the libraries
import pandas as pd
import html
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.sonority_sequencing import SyllableTokenizer
tk = SyllableTokenizer()
import nltk
nltk.download('stopwords')
nltk.download('punkt')

def flatten(l):
    return [item for sublist in l for item in sublist]

def preprocessor(df):

  for i in range (len(df)):
    x = df['text'][i].replace('\n',' ') #cleaning newline “\n” from the tweets
    df['text'][i] = html.unescape(x)

  for i in range (len(df)):
    df['text'][i] = re.sub(r'<br /><br />|(@[A-Za-z0–9_]+)|(#[A-Za-z0–9_]+)|[^\w\s]|http\S+', ' ', df['text'][i]) # add removal items for <br /><br /> and #something
  
  tweets_to_token = df['text']
  sw = stopwords.words('english') #you can adjust the language as you desire
  sw.remove('not') #we exclude not from the stopwords corpus since removing not from the text will change the context of the text

  return tweets_to_token

def tokenise(tweets_to_token):
  for i in range(len(tweets_to_token)):
    tweets_to_token[i] = word_tokenize(tweets_to_token[i]) # do the word tokenize

  for token in tweets_to_token[i]:
    tweets_to_token[i] = tk.tokenize(token)
    flatten(tweets_to_token[i])

  for i in range(len(tweets_to_token)):
    tweets_to_token[i] = ' '.join([word for word in tweets_to_token[i] if not word in sw]) # turn the tokenized listf into string to fit the format for applying CountVectorizer()
  
  return tweets_to_token


# read the data
cols = ['sentiment','id','date','query_string','user','text']
df_train = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin-1',header=None,names=cols)
print('Finish read train data')
df_test = pd.read_csv('testdata.manual.2009.06.14.csv', encoding='latin-1',header=None,names=cols)
print('Finish read test data')

# eliminated un-needed columns 
df_train.drop(['id','date','query_string','user'],axis=1,inplace=True)
print('Finish drop train data')
df_test.drop(['id','date','query_string','user'],axis=1,inplace=True)
print('Finish drop test data')

#start preprocessing
train_token = preprocessor(df_train)
train_token['sentiment'] = df_train['sentiment']#combine data with sentiment label to enable parallel without messy the data
train_token.to_csv('train_preprocessed.csv', index = False)
print('Finish preprocess train data')
test_token = preprocessor(df_test)
test_token['sentiment'] = df_test['sentiment']
test_token.to_csv('test_preprocessed.csv', index = False)
print('Finish preprocess test data')


Training

In [ ]:
#Reference from: https://stackoverflow.com/questions/28396957/sklearn-tfidf-vectorizer-to-run-as-parallel-jobs
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import multiprocessing
from multiprocessing import Pool
import scipy.sparse as sp
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

filename = 'svm_model.pkl'
num_cores = multiprocessing.cpu_count()
num_partitions = num_cores-2 # I like to leave some cores for other
#processes
print(num_partitions)
def parallelize_dataframe(df, func):
    a = np.array_split(df, num_partitions)
    del df
    pool = Pool(num_cores)
    df = sp.vstack(pool.map(func, a), format='csr')
    pool.close()
    pool.join()
    return df

def test_func(data):
    tfidf_matrix = vectorizer.transform(data["text"])
    return tfidf_matrix

# read the preprocess data
train_token = pd.read_csv('train_preprocessed.csv')
#train_token = train_token.join(df_train['sentiment'])
#print(train_token.columns)
#train_token.to_csv('train_preprocessed.csv', index=False)
print('Finish reading preprocessed train data')
test_token = pd.read_csv('test_preprocessed.csv')
test_token = test_token[test_token.sentiment != 2.0]# drop the class 2, since there is no class 2 in the testing dataset, so we won't get class 2 however hard we tried. Drop it to get the better result for test. 
#test_token = test_token.join(df_test['sentiment'])
#test_token.to_csv('test_preprocessed.csv', index=False)
print('Finish reading preprocessed test data')
#print(train_token.head(4))
'''
# let the vectorizer fit the data
vectorizer = TfidfVectorizer()#max_features=10000
X_train = vectorizer.fit(train_token['text'].values)
with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
print('Finish vectorize train data')
'''
#vectorize the data
with open('vectorizer.pkl', 'rb') as file:
  vectorizer = pickle.load(file)
X_test = vectorizer.transform(test_token['text'].values).toarray()
print('Finish vectorize test data')
#X_train = vectorizer.transform(train_token['text'].values.tolist()).toarray()
X_train = parallelize_dataframe(train_token, test_func)
print('Finish vectorize save')

#start to do grid search
#################### MultinomialNB #######################
'''param={'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000], 'fit_prior': [True, False]}
pre_model = MultinomialNB() #svm paramters tuning
model = GridSearchCV(pre_model,
                      param_grid=param,
                      scoring='accuracy',
                      cv=5,
                      return_train_score=True)'''
##################################################################
#################### svm #######################
'''param={'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'], 
    'degree': [1,3,5,7,9], 
    'gamma': ['scale', 'auto'], 
    'shrinking': [True, False], 
    'probability': [True, False],
    'verbose':[True, False]}
param={'kernel': ['linear']}
pre_model = svm.SVC() #svm paramters tuning
model = GridSearchCV(pre_model,
                      param_grid=param,
                      scoring='accuracy',
                      cv=5,
                      return_train_score=True)'''
model = svm.SVC()
##################################################################
#################### RF #######################
'''param={'n_estimators': [100, 200]}
pre_model = RandomForestClassifier()
model = GridSearchCV(pre_model,
                      param_grid=param,
                      scoring='accuracy',
                      cv=5,
                      return_train_score=True)'''
#model = RandomForestClassifier()
##################################################################
#################### SGD #######################
'''pre_model = SGDClassifier()
param = dict(
    loss=['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
    learning_rate=['optimal', 'invscaling', 'adaptive'],
    eta0=uniform(loc=1e-7, scale=1e-2)
)
print = RandomizedSearchCV(
    estimator=pre_model,
    param_distributions=param,
    cv=5,
    n_iter=50
)'''
#model = SGDClassifier()
##################################################################
#################### bnb #######################
#model = BernoulliNB()
'''param={'alpha': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000], 'fit_prior': [True, False]}
pre_model = BernoulliNB() #svm paramters tuning
model = GridSearchCV(pre_model,
                      param_grid=param,
                      scoring='accuracy',
                      cv=5,
                      return_train_score=True)'''
##################################################################
#################### logistic #######################
param={'penalty': ['l1', 'l2', 'elasticnet', 'none'], 'C': [0.00001, 0.0001, 0.001, 0.1, 1, 10, 100,1000]}
pre_model = LogisticRegression() #svm paramters tuning
model = GridSearchCV(pre_model,
                      param_grid=param,
                      scoring='accuracy',
                      cv=5,
                      return_train_score=True)
##################################################################
model.fit(X_train, train_token['sentiment'])
print('Finish train train data')
'''print(model.best_params_)
print(model.best_estimator_)
print(model.best_score_)'''

# save best model
with open(filename, 'wb') as file:
  joblib.dump(model, file, compress=1)
print('Finish save model')

Testing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import multiprocessing
from multiprocessing import Pool
import scipy.sparse as sp
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

filename = 'rf_model.pkl'
num_cores = multiprocessing.cpu_count()
num_partitions = num_cores-2 # I like to leave some cores for other
#processes
print(num_partitions)
def parallelize_dataframe(df, func):
    a = np.array_split(df, num_partitions)
    del df
    pool = Pool(num_cores)
    df = sp.vstack(pool.map(func, a), format='csr')
    pool.close()
    pool.join()
    return df

def test_func(data):
    tfidf_matrix = vectorizer.transform(data["text"])
    return tfidf_matrix

# read the preprocess data
train_token = pd.read_csv('train_preprocessed.csv')
print('Finish reading preprocessed train data')
test_token = pd.read_csv('test_preprocessed.csv')
test_token = test_token[test_token.sentiment != 2.0]# drop the class 2, since there is no class 2 in the testing dataset, so we won't get class 2 however hard we tried. Drop it to get the better result for test. 
print('Finish reading preprocessed test data')

#vectorize the data
with open('vectorizer.pkl', 'rb') as file:
  vectorizer = pickle.load(file)
X_train = parallelize_dataframe(train_token, test_func)
print('Finish vectorize train data')
X_test = vectorizer.transform(test_token['text'].values).toarray()
print('Finish vectorize test data')

#load files
model = joblib.load(filename)

# print the result
print('train')
predictions = model.predict(X_train)
print(predictions.shape, train_token['sentiment'].shape)
print(accuracy_score(train_token['sentiment'], predictions))
print(classification_report(train_token['sentiment'],predictions))

print('test')
predictions = model.predict(X_test)
print(accuracy_score(test_token['sentiment'], predictions))
print(classification_report(test_token['sentiment'],predictions))

Predict our data

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import multiprocessing
from multiprocessing import Pool
import scipy.sparse as sp
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import os
from pathlib import Path 

model_type = 'nb'
filename = model_type+'_model.pkl'
if not os.path.exists(model_type):
    Path(model_type).mkdir(parents=True, exist_ok=True)
num_cores = multiprocessing.cpu_count()
num_partitions = num_cores-2 # I like to leave some cores for other
#processes
print(num_partitions)
def parallelize_dataframe(df, func):
    a = np.array_split(df, num_partitions)
    del df
    pool = Pool(num_cores)
    df = sp.vstack(pool.map(func, a), format='csr')
    pool.close()
    pool.join()
    return df

def test_func(data):
    tfidf_matrix = vectorizer.transform(data["tweets_to_token"])
    return tfidf_matrix

#vectorize the data
with open('vectorizer.pkl', 'rb') as file:
    vectorizer = pickle.load(file)
#load files
model = joblib.load(filename)
# read the preprocess data
class_lst = ['barbera', 'cabernet', 'chardonnay', 'merlot', 'montepulciano', 'moscato', 'pinot_grigio', 'sangiovese', 'sauvignon', 'syrah', 'ciu_ciu_1', 'ciu_ciu_2', 'ciu_ciu_3', 'ciu_ciu_4']
for c in class_lst:
    test_token = pd.read_csv('token'+'/'+c+'_token.csv')
    #test_token.drop(['id','text','time','user_location','username','replyCount','retweetCount','likeCount','quoteCount','class'],axis=1,inplace=True)
    test_token.tweets_to_token=test_token.tweets_to_token.astype(str)
    #print('Finish reading preprocessed test data')

    X_test = parallelize_dataframe(test_token, test_func)
    #print('Finish vectorize test data')

    predictions = model.predict(X_test)
    df2 = pd.DataFrame(predictions, columns=['prediction'])
    df = pd.concat([test_token, df2], axis=1)
    df.to_csv(model_type+'/'+c+'_predict.csv', index=False)
    #print(df[df.prediction==0])
    del df
    del df2
    del test_token
    del X_test
    print(c)


Do some analyze on our data

In [ ]:
import pandas as pd
#pd.set_option("display.max_rows", None)

class_lst = ['Barbera', 'Cabernet', 'Chardonnay', 'Merlot', 'Montepulciano', 'Moscato', 'Pinot Grigio', 'Sangiovese', 'Sauvignon', 'Syrah', 'Ciù Ciù', 'CiuCiu', 'Ciu Ciu', 'CiùCiù']

df = []
for c in class_lst:
    df.append(pd.read_csv(c+'.csv', lineterminator='\n'))

res = pd.concat(df,axis=0, ignore_index=True)
# counting unique items
item_counts = res["id"].value_counts()
#print(item_counts.keys())
print(item_counts.value_counts()) #103515/625393 #8525
#item_counts[:103515].plot(kind='barh')

dupli = res[res.duplicated(subset=['id'], keep=False)]
dic = {}
#n = 0
for c in class_lst:
    dic[c] = 0
for k in item_counts.keys():
    sub = dupli[dupli['id']==k]
    keyword = sorted(list(sub.groupby(['class']).groups.keys()))
    if len(keyword) == 1:
        continue
    key =' '.join(keyword)
    if key not in dic:
        dic[key] = 1
    else:
        dic[key] += 1
    '''for i in range(len(sub)):
        dic[sub['class'].iloc[i]] += 1'''
#print(dupli.iloc[0])
#print(res.loc[res[dupli] == True & res['class'] == 'Barbera'])
#print(dic)
dic_sort = {k: v for k, v in sorted(dic.items(), key=lambda item: item[1])}
print(dic_sort)
'''{'Barbera': 725, 'Cabernet': 84491, 'Chardonnay': 17786, 'Merlot': 21596,
 'Montepulciano': 691, 'Moscato': 4111, 'Pinot Grigio': 4376, 'Sangiovese': 2474, 
 'Sauvignon': 78323, 'Syrah': 6315, 'Ciù Ciù': 323, 'CiuCiu': 40, 'Ciu Ciu': 323, 
 'CiùCiù': 40}'''

